In [1]:
!pip install boto3

You should consider upgrading via the 'C:\Users\Bryan\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install load_dotenv

You should consider upgrading via the 'C:\Users\Bryan\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import boto3
import warnings
import os
from dotenv import load_dotenv
warnings.filterwarnings("ignore")

In [2]:
load_dotenv('env.s3creds')

True

s3 = boto3.resource(
	service_name='s3',
	aws_access_key_id=os.getenv('aws_access_key_id'),
    aws_secret_access_key=os.getenv('aws_secret_access_key'),
    region_name=os.getenv('region_name')
)

In [3]:
s3 = boto3.resource(
	service_name='s3',
	aws_access_key_id=os.getenv('aws_access_key_id'),
    aws_secret_access_key=os.getenv('aws_secret_access_key'),
    region_name=os.getenv('region_name')
)

In [4]:
# Verify that the variables are correctly retrieved (optional)
print(f"Client Access: {s3}")


Client Access: s3.ServiceResource()


In [5]:
# List all buckets
for bucket in s3.buckets.all():
    print(bucket.name)

prj4


In [6]:
for obj in s3.Bucket('prj4').objects.all():
    print(obj)

s3.ObjectSummary(bucket_name='prj4', key='StormEvents_2014.csv')
s3.ObjectSummary(bucket_name='prj4', key='eaglei_outages_2014.csv')
s3.ObjectSummary(bucket_name='prj4', key='stormOutages_df.csv')


In [7]:
obj = s3.Bucket('prj4').Object('stormOutages_df.csv').get()
file = pd.read_csv(obj['Body'], index_col=0)

df = pd.DataFrame(file)

In [8]:
df.head(5)


,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out
END_YEARMONTH,,,,,,,
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52


In [9]:
df = df.assign(fips_code=df.index)

In [10]:
df.head()

,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out,fips_code
END_YEARMONTH,,,,,,,,
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411
201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411


In [11]:
df = df.reset_index(drop=True)

In [12]:
df.shape

(1007790, 8)

In [13]:
customer_counts = df['customers_out'].value_counts()

# Print the result
print(customer_counts)

1        261368
2         91848
3         48512
4         35958
5         32465
          ...  
10674         1
4032          1
4010          1
4004          1
4196          1
Name: customers_out, Length: 7944, dtype: int64


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007790 entries, 0 to 1007789
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   EVENT_ID       1007790 non-null  int64 
 1   STATE          1007790 non-null  object
 2   county         1007790 non-null  object
 3   YEAR           1007790 non-null  int64 
 4   EVENT_TYPE     1007790 non-null  object
 5   FIPS           1007790 non-null  int64 
 6   customers_out  1007790 non-null  int64 
 7   fips_code      1007790 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 61.5+ MB


In [15]:
df.columns

Index(['EVENT_ID', 'STATE', 'county', 'YEAR', 'EVENT_TYPE', 'FIPS',
       'customers_out', 'fips_code'],
      dtype='object')

In [16]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

### Step 3: Split the data into training and testing datasets by using `train_test_split`.

Due to the dataset containing many rows of value "1" for the amount of customers out, that value cannot be used to train in the data set. We will clean the data to remove the counts of "1" for customers out in the data. 

Create the labels set (`y`)  from the customers out value column, and then create the features (`X`) DataFrame from the remaining columns.

In [17]:
df_filtered = df.query("customers_out > 10")
df_filtered.head()

,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out,fips_code
0,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411
1,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411
2,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411
3,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411
4,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411


In [18]:
df_filtered['fips_code'] = df_filtered['fips_code'].astype(str)
df_filtered['month'] = df_filtered['fips_code'].str[-2:]
df_filtered.head()

,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out,fips_code,month
0,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411,11
1,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411,11
2,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11
3,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11
4,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11


### Step 2: Create the labels set (`y`)  from the “customers_out” column, and then create the features (`X`) DataFrame from the remaining columns.

### we need to convert string/char data types to a coded numerical value in order to translate the data into a usable format for logistical regression. 
## we will take all pertinent data such in str format such as "STATE" and "county" then concatenate them so they are unique.  -- NVM We can use FIPS !! 
# ## once they are merged, we can assign it a numerical value in alphabetical order to render in the model as a "Coded" location based value. --AGAIN  -- NVM We can use FIPS !! 
## EVENT_ID is garbage probably, DELETE 
### EVENT_TYPE Should be classified by intensity of the storm type in ASC order. yay :) 

In [19]:
event_type_counts = df['EVENT_TYPE'].value_counts()
event_type_counts

Winter Weather             205698
Thunderstorm Wind          133259
Winter Storm               106094
Flood                       88426
Strong Wind                 76272
Heavy Snow                  60019
Heavy Rain                  52016
Flash Flood                 50925
Dense Fog                   41227
High Wind                   38363
Drought                     33961
Hail                        26071
Lake-Effect Snow            24936
Tornado                     17513
Cold/Wind Chill              9538
Frost/Freeze                 8476
Funnel Cloud                 8263
Lightning                    7851
Coastal Flood                7829
Debris Flow                  3393
Seiche                       2997
Ice Storm                    2806
Wildfire                     1518
Extreme Cold/Wind Chill       279
Avalanche                      50
Blizzard                        8
Sleet                           2
Name: EVENT_TYPE, dtype: int64

In [20]:
# Define a dictionary to map weather events to scores
event_scores = {
    "Winter Weather": 2,  # Moderate impact
    "Thunderstorm Wind": 5,  # Can cause damage
    "Winter Storm": 7,  # Significant disruptions
    "Flood": 8,  # Major infrastructure damage
    "Strong Wind": 4,  # Potential for damage
    "Heavy Snow": 6,  # Disruptions to travel and infrastructure
    "Heavy Rain": 5,  # Can cause flooding and disruptions
    "Flash Flood": 8,  # Life-threatening and destructive
    "Dense Fog": 3,  # Can cause travel delays and accidents
    "High Wind": 4,  # Potential for damage
    "Drought": 7,  # Long-term impact on agriculture and water resources
    "Hail": 5,  # Can damage property and crops
    "Lake-Effect Snow": 6,  # Can cause heavy snowfall in localized areas
    "Tornado": 9,  # Devastating impact on property and lives
    "Cold/Wind Chill": 3,  # Discomfort and health risks for vulnerable populations
    "Frost/Freeze": 2,  # Can damage crops and vegetation
    "Funnel Cloud": 1,  # Not always associated with a tornado
    "Lightning": 3,  # Can cause fires and injuries
    "Coastal Flood": 8,  # Damage to coastal infrastructure and property
    "Debris Flow": 8,  # Rapid movement of debris can be destructive
    "Seiche": 2,  # Water level fluctuations, typically less impactful
    "Ice Storm": 7,  # Power outages and damage to infrastructure
    "Wildfire": 9,  # Widespread destruction of property and ecological damage
    "Extreme Cold/Wind Chill": 8,  # Life-threatening conditions
    "Avalanche": 9,  # Destructive and can cause fatalities
    "Blizzard": 8,  # Severe travel disruptions and potential infrastructure damage
    "Sleet": 3  # Mix of rain and snow, can cause slippery conditions
}

# Assign scores to the "EVENT_TYPE" column and create a new "domain" column
df_filtered["storm_intensity"] = df_filtered["EVENT_TYPE"].apply(lambda event: event_scores.get(event, 1))  # Assign 1 for missing values

# Print the modified DataFrame (optional)
df_filtered.head()

,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out,fips_code,month,storm_intensity
0,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411,11,5
1,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50,201411,11,5
2,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11,5
3,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11,5
4,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52,201411,11,5


In [21]:
# Separate the data into labels and features

# Separate the y variable, the labels
y=df_filtered['customers_out']

# Separate the X variables from the features
x = df_filtered.drop(columns=['customers_out', 'EVENT_ID', 'fips_code', 'STATE', 'county', 'EVENT_TYPE'])

In [22]:
y.head()

0    50
1    50
2    52
3    52
4    52
Name: customers_out, dtype: int64

In [23]:
y.value_counts()

11      21800
12      12261
13      10230
14       8569
15       8069
        ...  
3631        1
2920        1
2921        1
1727        1
4196        1
Name: customers_out, Length: 7934, dtype: int64

In [24]:
x.head()

,YEAR,FIPS,month,storm_intensity
0,2014,45079201411,11,5
1,2014,45079201411,11,5
2,2014,45079201411,11,5
3,2014,45079201411,11,5
4,2014,45079201411,11,5


### Step 3: Split the data into training and testing datasets by using `train_test_split`.

In [25]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split
### We had to remove the stratify = y argument due to multiple values in our testing data of having only one occurrence. The testing/training of this method
### requires multiple occurrences of the customer out value to train against, and one example will not allow the tool to analyze and understand the difference and 
## potential values to expect given the other values/parameters of our "X" data 
# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1)
X_train.shape

(338284, 4)

In [26]:
y_train.shape

(338284,)

In [27]:
y_test.shape

(112762,)

## Create a Logistic Regression Model with the Original Data
#####  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [28]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
Logistic_regr = LogisticRegression(solver='lbfgs',max_iter=200,random_state=1)
Logistic_regr

# Fit the model using training data
Logistic_regr.fit(X_train,y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [ ]:
# Make a prediction using the testing data
prediction = Logistic_regr.predict(X_test)
prediction_df = pd.DataFrame({"Prediction": prediction, "Actual": y_test}).reset_index(drop=True)
prediction_df.head()

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

### Step 3: Evaluate the model’s performance by doing the following:

* Generate a confusion matrix.

* Print the classification report.

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pandas as pd

In [ ]:
# Generate a confusion matrix for the model
conf_mat = confusion_matrix(y_test, prediction)
conf_mat_df = pd.DataFrame(conf_mat, index= ["Base 0 ", "Base 1 "], columns=["Predicted 0", "Predicted 1"])
accuracy_score = accuracy_score(y_test, prediction)

NameError: name 'prediction' is not defined

In [ ]:
from sklearn.metrics import classification_report

# Assuming model is your trained model and X_test is your test dataset
y_true = y_test  # True labels
y_pred = Logistic_regr.predict(X_test)  # Predicted labels

# Ensure y_pred is an array
y_pred = y_pred.tolist() if not isinstance(y_pred, (list, np.ndarray)) else y_pred

# Print the classification report for the model
# Displaying results
print("Confusion Matrix Chart")
display(conf_mat_df)
print(f"Accuracy Score : {accuracy_score}")
print("Classifications")
report = classification_report(y_true, y_pred)
print(report)

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** 

The model predicts healthy loans almost perfectly. It performs well for high-risk loans, but not as accurately. There are some false positives and false negatives in the high risk loan model, showing a drop in precision and recall, compared to healthy loans. Overall, the model is highly effective, but it could improve in reducing errors for high-risk loans.